## In this Jupyter Notebook we will display the results after training of the model

##### Setting up the conda environment

In [2]:
!conda env create -f environment.yml
!conda activate liver-segmentation


EnvironmentFileNotFound: 'C:\Users\vlad_cristian.luta\PycharmProjects\liver-and-tumor-segmentation\environment.yml' file not found



Failed to run 'conda activate liver-segmentation'.



EnvironmentNameNotFound: Could not find conda environment: liver-segmentation
You can list all discoverable environments with `conda info --envs`.




##### Importing the libraries

In [ ]:
import os
from glob import glob
import shutil
import dicom2nifti
import nibabel as nib
import numpy as np

from scripts.prepare import divide_groups, convert_nif, remove_empty_groups

##### Setting the path to the working directories

In [ ]:
# The input paths for the dicom files
in_path = ['data_set/dicom_files_testing/images',
            'data_set/dicom_files_testing/labels',
            'data_set/dicom_files_training/images',
            'data_set/dicom_files_training/labels',
            'data_set/dicom_files_validation/images',
            'data_set/dicom_files_validation/labels']

# The output paths for the dicom files split into equal groups
group_path = ['data_set_group/dicom_files_testing/images',
            'data_set_group/dicom_files_testing/labels',
            'data_set_group/dicom_files_training/images',
            'data_set_group/dicom_files_training/labels',
            'data_set_group/dicom_files_validation/images',
            'data_set_group/dicom_files_validation/labels']

# The output paths for the nifti files
nif_path = ['data_set_group_nif/nif_files_testing/images',
            'data_set_group_nif/nif_files_testing/labels',
            'data_set_group_nif/nif_files_training/images',
            'data_set_group_nif/nif_files_training/labels',
            'data_set_group_nif/nif_files_validation/images',
            'data_set_group_nif/nif_files_validation/labels']

##### Divide pacients into groups (prepare data)

In [ ]:
divide_groups(in_path, group_path, 16)

convert_nif(group_path, nif_path)

remove_empty_groups(nif_path)

##### Divide pacients into groups (prepare data)

##### Loading the model

In [ ]:
# Load the model
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
).to(device)

model.load_state_dict(torch.load(os.path.join(model_path, 'best_metric_model.pth')))

# Set the model to evaluation mode
model.eval()